In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# load all of the audio data 
fp = r"..\Data\csv\training_data.csv"
df = pd.read_csv(fp).drop("Unnamed: 0", axis = 1)
df.head()


,path,filename,full_duration,duration_limit,padding,missing_samples,spectrogram_min,spectrogram_max,mel_spectrogram_min,mel_spectrogram_max,mfcc_min,mfcc_max,spectrogram_path,mel_spectrogram_path,mfcc_path
0,E:\Sample library\2021\[WU] Hat - Bowl.wav,[WU] Hat - Bowl.wav,0.0,8,True,350879,-49.120300,30.879700,-74.402550,5.597448,-651.845215,93.570595,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
1,E:\Sample library\2021\[WU] Clap - Twist.wav,[WU] Clap - Twist.wav,0.0,8,True,348991,-37.832912,42.167088,-57.490067,22.509935,-510.737946,132.957932,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
2,E:\Sample library\2021\[WU] Perc (30).wav,[WU] Perc (30).wav,0.0,8,True,351196,-53.774376,26.225622,-74.140091,5.859912,-667.241821,34.692505,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
3,E:\Sample library\2021\[WU] Perc (22).wav,[WU] Perc (22).wav,0.0,8,True,349933,-39.270737,40.729263,-59.228420,20.771580,-481.627411,134.089661,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
4,E:\Sample library\2021\[WU] Open Hat - Tech.wav,[WU] Open Hat - Tech.wav,0.0,8,True,335821,-46.321472,33.678528,-71.892166,8.107837,-608.958801,87.468155,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...


In [4]:
# convert the entire column to lowercase and remove .wav from all samples
df['path'] = df['path'].apply(lambda x: x.lower()).str.replace(".wav","")
df.head() 

,path,filename,full_duration,duration_limit,padding,missing_samples,spectrogram_min,spectrogram_max,mel_spectrogram_min,mel_spectrogram_max,mfcc_min,mfcc_max,spectrogram_path,mel_spectrogram_path,mfcc_path
0,e:\sample library\2021\[wu] hat - bowl,[WU] Hat - Bowl.wav,0.0,8,True,350879,-49.120300,30.879700,-74.402550,5.597448,-651.845215,93.570595,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
1,e:\sample library\2021\[wu] clap - twist,[WU] Clap - Twist.wav,0.0,8,True,348991,-37.832912,42.167088,-57.490067,22.509935,-510.737946,132.957932,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
2,e:\sample library\2021\[wu] perc (30),[WU] Perc (30).wav,0.0,8,True,351196,-53.774376,26.225622,-74.140091,5.859912,-667.241821,34.692505,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
3,e:\sample library\2021\[wu] perc (22),[WU] Perc (22).wav,0.0,8,True,349933,-39.270737,40.729263,-59.228420,20.771580,-481.627411,134.089661,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...
4,e:\sample library\2021\[wu] open hat - tech,[WU] Open Hat - Tech.wav,0.0,8,True,335821,-46.321472,33.678528,-71.892166,8.107837,-608.958801,87.468155,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...,E:\Documents\My Projects\Filtr\Data\audio_vect...


In [173]:
def create_pattern(words):
    ''' used to create single string that can be used by the 
        .str.contains() method to find rows using a list of words.'''

    # make sure words is a list and if not convert it to one
    words = words.split() if isinstance(words, str) else words
    if len(words) <= 1:
        return "".join(words)
    else:
        return "|".join(words)

# return all of the data that contains the pattern
def contains(words, frame):
    # make sure a dataframe is being passed
    assert isinstance(frame, pd.core.frame.DataFrame), f"Frame parameter must be a pandas dataframe, not {type(frame)}"
    frame = frame.copy()
    # combine multiple words into a single pattern (if applicable)
    pattern = create_pattern(words)
    # filter the dataframe to return a new series with all of the rows that contain the pattern
    match_mask = frame['path'].str.contains(pattern)
    matched_frame = frame[match_mask]['path'].str.replace(pattern, " ")
    return matched_frame

def trim_path_directories(path: str):
    ''' removes the first 3 directories from the path if the file is in the e: drive 
        and removes the first 4 directories from the path if the file is in the d: drive '''
    
    # returns the path split by directory 
    directories = path.split("\\")
    drive = directories[0]
    # print(drive)
    if drive == "d:":
        return directories[4:]
    elif drive == "e:":
        return directories[3:]

def clean(docs: list):
    ''' cleans a document and return tokens as a string'''
    # convert the list into a string
    doc = " ".join(docs)
    # replace dashes with empty string
    doc = doc.replace("-", "")
    doc = doc.replace("_", " ")
    # remove punctuation 
    table = str.maketrans(dict.fromkeys(string.punctuation))
    doc = doc.translate(table)
    # remove numbers
    remove_digits = str.maketrans(dict.fromkeys(string.digits))
    doc = doc.translate(remove_digits)
    # create tokens
    doc = doc.split()
    # remove stop words and words shorter than 3 characters
    stops = set(nltk.corpus.stopwords.words("english"))
    doc = [word for word in doc if word not in stops and len(word) > 2]
    # get only the unique tokens
    doc = set(doc)
    # remove 's' at the end of words
    doc = [word[:-1] for word in doc if word.endswith('s')]
    # return as a string
    doc = " ".join(doc)
    return doc

def get_document(path: str):
    ''' takes in a path and returns it as a cleaned document '''

    # cleanup the text used to represent the directories
    directories = trim_path_directories(path)
    document = clean(directories)
    return document

def vectorize(documents: list, algorithm = "count", **kwargs):
    ''' takes in a list of documents and vectorizes them based on the algorithm '''
    algorithm = algorithm.lower()

    # optional keyword arguments
    binary = kwargs.get("binary", False)
    ngram_range = kwargs.get("ngram_range", (1, 1))
    max_df = kwargs.get("max_df", 1.0)
    min_df = kwargs.get("min_df", 1)


    # make sure a valid algorithm is being used
    assert (algorithm == 'count' or algorithm == "tfidf"), f"Algorithm must be 'count' or 'tfidf' not: {algorithm}"

    if algorithm == "tfidf":
        vectorizer = TfidfVectorizer(binary = binary,
                                     ngram_range = ngram_range,
                                     max_df = max_df,
                                     min_df = min_df)
    else:
        vectorizer = CountVectorizer(binary=binary,
                                     ngram_range=ngram_range,
                                     max_df=max_df,
                                     min_df=min_df)

    vectorizer.fit_transform(documents)
    return vectorizer

def get_word_probabilities(documents: list):
    ''' gets the probabilities of a word appearing given a filter word '''
    counter = Counter()
    # add each of the words in all documents to the counter
    # counter.update([doc.split() for doc in documents])
    for document in documents:
        document = document.split() 
        counter.update(document)
    
    counter = [(word, counter[word] / len(documents) * 100.0) for word in counter]
    return counter
    


words = ["hello","my", "friend"]
data = contains('kick snare', df)
data = data.apply(lambda x: get_document(x))
data.shape


(3734,)

In [174]:
# vector = vectorize(data.tolist(), algorithm = 'tfidf')
# Counter(vector.vocabulary_).most_common()
# vector.get_feature_names()
# vector.get_params()


def by_values(values):
    return values[1]

data.tolist()
values = get_word_probabilities(data.tolist())
values.sort(key = by_values, reverse = True)
values


[('kit', 34.09212640599893),
 ('drum', 28.334226031065878),
 ('hit', 14.702731655061596),
 ('shot', 14.086770219603642),
 ('cymatic', 13.711837171933583),
 ('sound', 10.765934654525978),
 ('clap', 7.90037493304767),
 ('loop', 5.677557579003749),
 ('bas', 5.168719871451526),
 ('oneshot', 4.177825388323514),
 ('sample', 3.8296732726298877),
 ('beat', 3.535083020889127),
 ('snap', 2.222817354043921),
 ('jview', 1.92822710230316),
 ('view', 1.92822710230316),
 ('electronic', 1.92822710230316),
 ('present', 1.6068559185859668),
 ('sp', 1.472951258703803),
 ('metrapoli', 1.44617032672737),
 ('wolve', 1.3926084627745046),
 ('at', 1.3658275307980718),
 ('state', 1.3658275307980718),
 ('legend', 1.1515800749866096),
 ('cosmic', 1.1247991430101767),
 ('loui', 1.098018211033744),
 ('futon', 1.098018211033744),
 ('stuff', 1.0712372790573113),
 ('general', 1.0444563470808785),
 ('sclap', 0.96411355115158),
 ('rim', 0.9373326191751473),
 ('vibe', 0.9373326191751473),
 ('ragubeat', 0.9373326191751473

In [131]:
v = 'hello'
v[:-1]

'hell'